In [1]:
import numpy as np
from models import load_models
from vector_database import get_reranked_documents
import torch

torch.set_default_device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
torch.cuda.is_available()

True

In [3]:
embedding_model, reranker_model = load_models()

In [4]:
polytrauma_queries = ['Haupttodesursache Polytraumapatient', 'Polytrauma Herzkranker Patient',
                      'Was muss man beim Wundmanagement beachten?']
kohlenmonoxyd_queries = ['Behandlung von Kohlenmonoxidvergiftung',
                         'Woran erkenne ich die schwere einer Kohlenmonoxydvergiftung?',
                         'Wann sollte eine Hyperbare Therapie eingesetzt werden?']
analgesie_queries = ['Welches Schmerzmittel sollte ich einem Kleinkind mit gebrochenen Knochen geben?',
                     'Schmerzeinschätzung bei ansprechbaren Patienten', 'Betäubungsmittel bei schwangerer Frau']

In [5]:
queries = polytrauma_queries + kohlenmonoxyd_queries + analgesie_queries

In [99]:
current_query = queries[5]

In [100]:
data = get_reranked_documents(current_query, embedding_model, reranker_model)
data[['reranking_score', 'cosine_similarity']]

,reranking_score,cosine_similarity
0,0.767690,0.386638
1,0.746139,0.457518
2,0.661923,0.547933
3,0.660172,0.496634
4,0.585101,0.352699


In [101]:
for _, text, heading, title, score in data[['text', 'heading', 'title', 'reranking_score']].itertuples():
    print(70 * '#')
    print(f'{heading} in: "{title}"\nScore: {score}')
    print(70 * '#' + '\n')
    print(f'**Anfrage: {current_query}**\n')
    print(text + '\n')

######################################################################
10 Hyperbare Sauerstofftherapie (HBOT) in: "S2k Diagnostik Therapie Kohlenmonoxidvergiftung"
Score: 0.7676898241043091
######################################################################

**Anfrage: Wann sollte eine Hyperbare Therapie eingesetzt werden?**

Wann besteht die Indikation für eine hyperbare Therapie?

######################################################################
10 Hyperbare Sauerstofftherapie (HBOT) in: "S2k Diagnostik Therapie Kohlenmonoxidvergiftung"
Score: 0.7461389899253845
######################################################################

**Anfrage: Wann sollte eine Hyperbare Therapie eingesetzt werden?**

Bei den Anzeichen einer schweren Kohlenmonoxidvergiftung (u.a. fortgesetzte Bewusstseinsstörungen, metabolische Azidose, respiratorische Insuffizienz und/oder kardiale Ischämie) sowie bei Schwangerschaft sollte im Erwachsenenalter (18 Jahre) eine hyperbare Sauerstofftherapie durc

In [94]:
feedbacks = [[1, 1, 1, 1, 1], [1, 0, 1, 0, 0], [1, 1, 1, 1 ,1], [1, 1, 1, 0, 1],[1, 0, 0, 1, 0], [0, 1, 0, 0, 1], [1, 1, 1, 0, 1], [1, 1, 1, 0, 1], [1, 1, 1, 1, 1]]

In [95]:
top_5_recalls = [sum(feedback[:5])/5 for feedback in feedbacks]
top_3_recalls = [sum(feedback[:3])/3 for feedback in feedbacks]
top_1_recalls = [sum(feedback[:1]) for feedback in feedbacks]

In [96]:
mean_top_5_recall = np.mean(top_5_recalls)
mean_top_3_recall = np.mean(top_3_recalls)
mean_top_1_recall = np.mean(top_1_recalls)

In [97]:
print(f'Average Top 5 Recall: {mean_top_5_recall:.4f}\nAverage Top 3 Recall: {mean_top_3_recall:.4f}\nAverage Top 1 Recall: {mean_top_1_recall:.4f}')

Average Top 5 Recall: 0.7333
Average Top 3 Recall: 0.8148
Average Top 1 Recall: 0.8889
